In [1]:
import findspark
findspark.init("/Users/chukuemekaogudu/Documents/Dev-Spark-Apache/Apache-Spark/spark-2.4.5-bin-hadoop2.7")

import os
import json
import time
from pyspark import SparkContext, SparkConf
import math

In [2]:
data_dir = "/Volumes/oli2/inf533_datasets"

In [3]:
config = SparkConf().setMaster("local[*]") \
                    .setAppName("Task2Opt") \
                    .set("spark.executor.memory", "4g") \
                    .set("spark.driver.memory", "4g")
sc = SparkContext(conf=config).getOrCreate()

In [4]:
model_dict = sc.textFile(os.path.join(data_dir, "output_user_based.json")) \
               .map(json.loads) \
               .map(lambda x: ((x["u1"], x["u2"]), x["sim"])) \
               .collectAsMap()

In [5]:
test_rdd = sc.textFile(os.path.join(data_dir, "test_review.json")) \
             .map(json.loads) \
             .map(lambda x: (x["business_id"], x["user_id"]))

In [6]:
avg_dict = sc.textFile(os.path.join(data_dir, "user_avg.json")) \
             .map(json.loads) \
             .map(dict) \
             .flatMap(lambda x: x.items()) \
             .collectAsMap()

In [7]:
train_rdd = sc.textFile(os.path.join(data_dir, "train_review.json")).map(json.loads).cache()

In [8]:
unique_id = train_rdd.map(lambda x: x["user_id"]).distinct().collect()

In [9]:
business_user = train_rdd.map(lambda x: (x["business_id"], (x["user_id"], x["stars"]))) \
                         .groupByKey() \
                         .mapValues(lambda x: list(set(x)))

In [10]:
N = 3

In [11]:
def prediction(data, avg_dict, id_list, sim_dict):
    target = data[0]
    if target not in id_list:
        target = "UNK"
    pairs = data[1]
    
    values = []
    for pair in pairs:
        sim = 0
        if sim_dict.get(tuple((pair[0], target))) != None:
            sim = sim_dict.get(tuple((pair[0], target)))
        if sim_dict.get(tuple((target, pair[0]))) != None:
            sim = sim_dict.get(tuple((target, pair[0])))
        values.append((sim, pair[1], avg_dict.get(pair[0], avg_dict["UNK"])))
        
    num = 0
    for v in values:
        num += (v[1] - v[2]) * v[0]
        
    denum = 0
    for v in values:
        denum += abs(v[0])
    if denum == 0 or num == 0:
        return (target, avg_dict.get(target, avg_dict["UNK"]))
    result = (num / float(denum)) + avg_dict.get(target, avg_dict["UNK"])
    return (target, result)

In [16]:
results = test_rdd.leftOuterJoin(business_user).filter(lambda x: x[1][1] != None) \
                  .mapValues(lambda x: prediction(x, avg_dict, unique_id, model_dict)) \
                  .collect()

In [19]:
results[:-100]

[('iCQpiavjjPzJ5_3gPD5Ebg', ('srm0YUaJubOLxs4ByEZpwg', 3.487141518889846)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('3dPjKulIKmW2NXUkrfgbHw', 4.380441349563997)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('pJUzdorsC5BpC3q5ECi4Lg', 4.251305720687609)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('Zqk-ye_OHeH_sS1JsdsLgg', 3.2235241698757697)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('MqcfaSdLXh7H1Aey_HPUsw', 4.334737504587771)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('FIDPP65G4FxtZfqt5HwWWg', 4.153549749994817)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('IS_CMUedfBmUxQk-pdehww', 4.3183180834655115)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('aQJtyvmZg3kCd-tfZ6PLeg', 4.0202858731834255)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('XXSDiRsswmzNa-J7pMYFfA', 4.457358062297096)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('AWlitAwFS-0KtCfyCWh0nQ', 3.375)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('c0VLfFUo2PRzCdiMfVZi3A', 3.8273288138730757)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('n-Hit0Y3O6FxTiEhdEEsZw', 4.453711185889228)),
 ('iCQpiavjjPzJ5_3gPD5Ebg', ('x2-TP8fd4G0OqkgDyKt3IA', 4.1966586216157955)),
 ('iCQpiavjjPzJ5_3g

In [15]:
sc.stop()

In [ ]:
.mapValues(lambda x: prediction(x, avg_dict, unique_id, model_dict)) \
                  .collect()